In [ ]:
### I cannot upload extract.py so i will paste the content here
def extract_json(response_text):
    import re
    import ast
    match = re.search(r'```json(.*?)```', response_text, re.DOTALL)
    if not match:
        return None
    json_like_str = match.group(1).strip()
    try:
        return ast.literal_eval(json_like_str)
    except (ValueError, SyntaxError) as e:
        print(f"Failed to parse dictionary: {e}")
        return None
    

def extract_dollars(response_text):
    import re
    import ast
    match = re.search(r'start\$(.*?)\$end', response_text, re.DOTALL)
    if not match:
        return None
    json_like_str = match.group(1).strip()
    try:
        return ast.literal_eval(json_like_str)
    except (ValueError, SyntaxError) as e:
        print(f"Failed to parse dictionary: {e}")
        return None



def get_next_question_id(pair_id, counter_path = r"D:\AStar\Code\Jupyter Notebook\question_id_counter.json"):
    import json
    import os

    if os.path.exists(counter_path):
        with open(counter_path, "r") as f:
            counters = json.load(f)
    else:
        counters = {}

    current = counters.get(pair_id, 0) + 1
    question_id = f"{pair_id}{current:02d}"

    counters[pair_id] = current
    with open(counter_path, "w") as f:
        json.dump(counters, f)

    return question_id


In [1]:
import pandas as pd
import numpy as np

In [2]:
all_sheets = pd.read_excel(r'D:\AStar\Code\Data\Excel\KCs_with_L1short.xlsx', sheet_name=None)
primary_df = all_sheets['Primary level']
secondary_df = all_sheets['O level']
secondary_df["Grade_level_1_ID"] = secondary_df['Grade_ID'] + secondary_df['Level_1_short']
secondary_df.head()

,Grade,Grade_ID,Sub_strand,Order,Exclude,Level_1,Level_1_short,Level_2_no,Level_2,KC,ID,Grade_level_1_ID
0,Secondary O-level 1,O1,NUMBER AND ALGEBRA,1,NaN,Numbers and their operations,NumOps,1,primes and prime factorisation,Numbers and their operations : primes and prim...,O1NumOps1,O1NumOps
1,Secondary O-level 1,O1,NUMBER AND ALGEBRA,2,NaN,Numbers and their operations,NumOps,2,finding highest common factor (HCF) and lowest...,Numbers and their operations : finding highest...,O1NumOps2,O1NumOps
2,Secondary O-level 1,O1,NUMBER AND ALGEBRA,3,NaN,Numbers and their operations,NumOps,3,"negative numbers, integers, rational numbers, ...",Numbers and their operations : negative number...,O1NumOps3,O1NumOps
3,Secondary O-level 1,O1,NUMBER AND ALGEBRA,4,NaN,Numbers and their operations,NumOps,4,calculations with calculator,Numbers and their operations : calculations wi...,O1NumOps4,O1NumOps
4,Secondary O-level 1,O1,NUMBER AND ALGEBRA,5,NaN,Numbers and their operations,NumOps,5,representation and ordering of numbers on the ...,Numbers and their operations : representation ...,O1NumOps5,O1NumOps


In [3]:
from itertools import combinations
same_grade_level_1_pairs = {}

for level_1_value, group in secondary_df.groupby('Grade_level_1_ID'):
    ids = group['ID'].tolist()  
    same_grade_level_1_pairs[level_1_value] = list(combinations(ids, 2))



In [4]:
same_grade_level_1_pairs_test = same_grade_level_1_pairs['O1NumOps']
same_grade_level_1_pairs_test

[('O1NumOps1', 'O1NumOps2'),
 ('O1NumOps1', 'O1NumOps3'),
 ('O1NumOps1', 'O1NumOps4'),
 ('O1NumOps1', 'O1NumOps5'),
 ('O1NumOps1', 'O1NumOps6'),
 ('O1NumOps1', 'O1NumOps7'),
 ('O1NumOps2', 'O1NumOps3'),
 ('O1NumOps2', 'O1NumOps4'),
 ('O1NumOps2', 'O1NumOps5'),
 ('O1NumOps2', 'O1NumOps6'),
 ('O1NumOps2', 'O1NumOps7'),
 ('O1NumOps3', 'O1NumOps4'),
 ('O1NumOps3', 'O1NumOps5'),
 ('O1NumOps3', 'O1NumOps6'),
 ('O1NumOps3', 'O1NumOps7'),
 ('O1NumOps4', 'O1NumOps5'),
 ('O1NumOps4', 'O1NumOps6'),
 ('O1NumOps4', 'O1NumOps7'),
 ('O1NumOps5', 'O1NumOps6'),
 ('O1NumOps5', 'O1NumOps7'),
 ('O1NumOps6', 'O1NumOps7')]

In [5]:
from extract import extract_dollars
from extract import extract_json
from extract import get_next_question_id

In [6]:
topics = ["animals", "foods", "transportation", "school", "family", "sports"]
temperatures = [0.2, 0.5, 0.8, 1.2]
models = {"gpt": "openai/gpt-4.1",
          "claude": "anthropic/claude-sonnet-4", 
          "deepseek": "deepseek/deepseek-r1", 
          "phi":"microsoft/phi-4", 
          "qwen": "qwen/qwen3-32b",
          "llama":"meta-llama/llama-4-scout"}
models_id = {"gpt": "1",
            "claude": "2", 
            "deepseek": "3", 
            "phi":"4",
            "qwen": "5",
            "llama":"6"}
model_evaluate_prompts = {"gpt": r"D:\AStar\Code\Prompts\GPT\evaluate_gpt.txt",
            "claude": r"D:\AStar\Code\Prompts\CLAUDE\evaluate_claude.txt", 
            "deepseek": r"D:\AStar\Code\Prompts\DEEPSEEK\evaluate_deepseek.txt", 
            "phi":r"D:\AStar\Code\Prompts\PHI\evaluate_phi.txt",
            "qwen": r"D:\AStar\Code\Prompts\QWEN\evaluate_qwen.txt",
            "llama": r"D:\AStar\Code\Prompts\LLAMA\evaluate_llama.txt"}

In [8]:
def convert_to_dict(response=None, **kwargs):
    # Add the response to the kwargs dictionary
    kwargs['response'] = response
    return kwargs

In [17]:
import ast
import os
from dotenv import load_dotenv
from openai import OpenAI
from string import Template

def question_generator_topic(model, prompt, qid, grade, kc1, kc2, topic):
    load_dotenv()
    TOKEN = os.getenv("Openrouter_TOKEN")
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=TOKEN,
    )

    with open(prompt, 'r', encoding='utf-8') as file:
        template = file.read()

    template = Template(template)
    filled_prompt = template.safe_substitute(qid = qid, grade = grade, kc1 = kc1, kc2 = kc2, topic = topic)
    messages =  ast.literal_eval(filled_prompt)
    response = client.chat.completions.create(
        model=models[model], 
        messages=messages,
        temperature = 0.6,
        n=1,
        max_tokens=10000,
        stop=None
    )
    response_text = response.choices[0].message.content
    response_dict = extract_dollars(response_text)
    if response_dict is None:
        response_dict = extract_json(response_text)
        if response_dict is None:
            try:
                response_dict = ast.literal_eval(response_text)
            except (ValueError, SyntaxError):
                with open("error.txt", 'a', encoding='utf-8') as file:
                    file.write(f"ID: {id}\n")
                    file.write(f"Prompt: {prompt}\n")
                    file.write(f"Response:\n{response_text}\n")
                    file.write(f"{'-'*40}\n")
                return None
    return response_dict

In [19]:
import json
model = "gpt"
prompt = r"D:\AStar\Code\Prompts\GPT\multiple_topic_temperature_gpt.txt"
grade = "Secondary O Level 1"
output_path = r"D:\AStar\Code\Data\Generation_multiple_KCs_steps\gpt_topic_1.json"
if os.path.exists(output_path):
    with open(output_path, "r", encoding="utf-8") as f:
        all_responses = json.load(f)
else:
    all_responses = {}
for id_kc1, id_kc2 in same_grade_level_1_pairs_test:
    kc1 = secondary_df.loc[secondary_df['ID'] == id_kc1, 'KC'].values[0]
    kc2 = secondary_df.loc[secondary_df['ID'] == id_kc2, 'KC'].values[0]
    for topic in topics:
        qid = id_kc1 + id_kc2 + topic + model 
        qid = get_next_question_id(qid)
        response = question_generator_topic(model = model, prompt = prompt, grade = grade, qid = qid, kc1 = kc1, kc2 = kc2, topic = topic)
        print(response)
        response = convert_to_dict(qid = qid, grade = grade, kc1 = kc1, kc2 = kc2, topic = topic, temperature = 0.6, difficulty = None, response = response)
        if response:  
            all_responses[qid] = response
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(all_responses, f, indent=2)

{'Question': 'A zoo is organising two animal shows: one featuring lions and the other featuring parrots. There are 72 lions and 90 parrots. The zookeeper wants to arrange both groups into identical groups such that each group contains the same number of animals of only one type, and no animal is left out. \n\n(a) Find the largest possible number of animals in each group. \n\n(b) Using prime factorisation, find the smallest number of animals that must be in a group if groups of lions and groups of parrots are to have the same number of animals, and each group must have at least one animal of each type.', 'Solution': '(a) To find the largest possible number of animals in each group (with only one type per group and no animal left out), we need to find the highest common factor (HCF) of 72 and 90.\n\nPrime factorisation:\n\n$72 = 2^3 \\times 3^2$\n\n$90 = 2^1 \\times 3^2 \\times 5^1$\n\nThe HCF is the product of the smallest powers of common primes:\n\n$\\text{HCF} = 2^1 \\times 3^2 = 2 \

<unknown>:2: SyntaxWarning: invalid escape sequence '\('


{'Question': 'During a mathematics class in school, Mr Tan asks his students to find the difference between the product of the two smallest prime numbers and the sum of the two largest negative integers less than \\(-5\\). What is the value of this difference?', 'Solution': 'First, identify the two smallest prime numbers: 2 and 3.\n\nThe product of the two smallest prime numbers is:\n\\[\n2 \\times 3 = 6\n\\]\n\nNext, find the two largest negative integers less than \\(-5\\). These are \\(-6\\) and \\(-7\\).\n\nThe sum of these two integers is:\n\\[\n(-6) + (-7) = -13\n\\]\n\nNow, find the difference between the product (6) and the sum (\\(-13\\)):\n\\[\n6 - (-13) = 6 + 13 = 19\n\\]\n\n\\textbf{Final Answer:} The value of the difference is \\(19\\).'}
{'Question': 'In the Lim family, Mr Lim wants to distribute a collection of 60 apples equally among his three children. Each child will receive the same number of apples. \n\n(a) Find the prime factorisation of 60.\n\n(b) After distributi

<unknown>:2: SyntaxWarning: invalid escape sequence '\c'


{'Question': 'Two buses depart from a bus terminal at the same time. Bus A completes a loop around the city in 36 minutes, while Bus B completes the same loop in 48 minutes. \n\n(a) After how many minutes will both buses be at the terminal together again? \n\n(b) If the temperature at the terminal was 3$^\\circ$C at the start, and it drops by 2$^\\circ$C each time both buses return together, what will the temperature be after they have returned together for the third time? \n\n(c) Find the cube root of the lowest common multiple (LCM) of 36 and 48 by prime factorisation.', 'Solution': 'Let us solve each part step by step.\n\n(a) To find when both buses will be at the terminal together again, we need to find the lowest common multiple (LCM) of 36 and 48.\n\nPrime factorisation:\n$36 = 2^2 \\times 3^2$\n$48 = 2^4 \\times 3^1$\n\nLCM is found by taking the highest power of each prime:\n$\\text{LCM}(36, 48) = 2^4 \\times 3^2 = 16 \\times 9 = 144$\n\nSo, both buses will be at the terminal t

...

{'Question': 'A wildlife reserve keeps track of the temperatures in two animal enclosures. The temperature in the penguin enclosure is $-3.75^\\circ$C, while the temperature in the reptile enclosure is $22.46^\\circ$C. \n\n(a) Find the difference in temperature between the two enclosures. Give your answer as a rational number. \n\n(b) Round your answer in part (a) to 1 decimal place. \n\n(c) Estimate the difference in temperature by rounding both temperatures to the nearest integer before finding the difference.', 'Solution': 'Let the temperature in the penguin enclosure be $T_p = -3.75^\\circ$C and in the reptile enclosure be $T_r = 22.46^\\circ$C.\n\n(a) The difference in temperature is:\n\\[\n\\text{Difference} = T_r - T_p = 22.46 - (-3.75) = 22.46 + 3.75 = 26.21^\\circ\\text{C}.\n\\]\n\n(b) Rounding $26.21$ to 1 decimal place:\n\\[\n26.21 \\approx 26.2^\\circ\\text{C}\n\\]\n\n(c) Rounding both temperatures to the nearest integer:\n\\[\nT_p \\approx -4^\\circ\\text{C}, \\quad T_r \\

<unknown>:2: SyntaxWarning: invalid escape sequence '\c'
<unknown>:3: SyntaxWarning: invalid escape sequence '\c'


{'Question': 'During a school science experiment, the temperature in a classroom was recorded as $-2.47^\\circ$C early in the morning. By noon, the temperature increased by $6.83^\\circ$C. \n\n(a) What was the temperature at noon? Give your answer as a rational number. \n\n(b) Round your answer in part (a) to 1 decimal place. \n\n(c) Estimate the temperature at noon by rounding the initial temperature and the increase to the nearest integer before adding.', 'Solution': 'Let the initial temperature be $-2.47^\\circ$C and the increase be $6.83^\\circ$C.\n\n(a) Temperature at noon:\n\\[\n-2.47 + 6.83 = 4.36\n\\]\nSo, the temperature at noon was $4.36^\\circ$C.\n\n(b) Rounded to 1 decimal place:\n\\[\n4.36 \\approx 4.4\n\\]\nSo, the temperature at noon to 1 decimal place is $4.4^\\circ$C.\n\n(c) Estimation by rounding to the nearest integer:\nInitial temperature: $-2.47 \\approx -2$\nIncrease: $6.83 \\approx 7$\nEstimated temperature at noon:\n\\[\n-2 + 7 = 5\n\\]\nTherefore, the estimated

<unknown>:2: SyntaxWarning: invalid escape sequence '\c'
<unknown>:3: SyntaxWarning: invalid escape sequence '\c'


{'Question': 'During a football match, the temperature at the start was $2.6^\\circ$C. By halftime, the temperature dropped by $3.8^\\circ$C. \n\n(a) What was the temperature at halftime? \n\n(b) Round your answer in part (a) to the nearest integer. \n\n(c) Estimate, to 1 decimal place, the total change in temperature from the start to halftime.', 'Solution': 'Let the temperature at the start be $2.6^\\circ$C.\n\n(a) The temperature dropped by $3.8^\\circ$C, so:\n\n\\[\n\\text{Halftime temperature} = 2.6 - 3.8 = -1.2^\\circ\\text{C}\n\\]\n\n(b) Rounded to the nearest integer:\n\n\\[\n-1.2 \\approx -1\n\\]\n\nSo, the temperature at halftime to the nearest integer is $-1^\\circ$C.\n\n(c) To estimate the total change in temperature to 1 decimal place:\n\nThe total change is $|2.6 - (-1.2)| = |2.6 + 1.2| = 3.8^\\circ$C.\n\nRounded to 1 decimal place, the estimated total change is $3.8^\\circ$C.'}
{'Question': 'A wildlife reserve is monitoring the weights of four different animals: an eleph

In [20]:
### Restructure the json file
'''
# Create a new dictionary to store the data with string keys for kc1, kc2 pairs
structured_data = {}

# Iterate over the data and transform it into the new structure
for key, value in data.items():
    kc1 = value['kc1']
    kc2 = value['kc2']
    topic = value['topic']
    temperature = value['temperature']
    
    kc_pair_key = f"{kc1} | {kc2}"
    
    if kc_pair_key not in structured_data:
        structured_data[kc_pair_key] = {}
    
    if topic not in structured_data[kc_pair_key]:
        structured_data[kc_pair_key][topic] = {}
    
    structured_data[kc_pair_key][topic][temperature] = value

# Define the output file path
output_file_path = '/mnt/data/restructured_data_corrected.json'

# Save the transformed data to a new JSON file
with open(output_file_path, 'w') as file:
    json.dump(structured_data, file, indent=4)

output_file_path
'''

'\n# Create a new dictionary to store the data with string keys for kc1, kc2 pairs\nstructured_data = {}\n\n# Iterate over the data and transform it into the new structure\nfor key, value in data.items():\n    kc1 = value[\'kc1\']\n    kc2 = value[\'kc2\']\n    topic = value[\'topic\']\n    temperature = value[\'temperature\']\n    \n    kc_pair_key = f"{kc1} | {kc2}"\n    \n    if kc_pair_key not in structured_data:\n        structured_data[kc_pair_key] = {}\n    \n    if topic not in structured_data[kc_pair_key]:\n        structured_data[kc_pair_key][topic] = {}\n    \n    structured_data[kc_pair_key][topic][temperature] = value\n\n# Define the output file path\noutput_file_path = \'/mnt/data/restructured_data_corrected.json\'\n\n# Save the transformed data to a new JSON file\nwith open(output_file_path, \'w\') as file:\n    json.dump(structured_data, file, indent=4)\n\noutput_file_path\n'

In [21]:
'''# Import the necessary libraries
import json

# Load the restructured JSON file
input_file_path = '/mnt/data/restructured_data_corrected.json'
with open(input_file_path, 'r') as file:
    structured_data = json.load(file)

# Define the output LaTeX file path
output_tex_path = '/mnt/data/questions_output.tex'

# Open the LaTeX file for writing
with open(output_tex_path, 'w') as tex_file:
    # Iterate over the data to create the LaTeX structure
    for kc_pair_key, topics in structured_data.items():
        # Write the first layer: KCs as header 1
        tex_file.write(f"\\section*{{{kc_pair_key}}}\n\n")
        
        for topic, temperatures in topics.items():
            # Write the second layer: Topic as header 2
            tex_file.write(f"  \\subsection*{{{topic}}}\n")
            
            for temperature, content in temperatures.items():
                # Write the third layer: Temperature as header 3
                tex_file.write(f"    \\subsubsection*{{Temperature: {temperature}}}\n")
                
                # Write the Question and Solution in LaTeX format
                tex_file.write("    \\textbf{Question} \n")
                tex_file.write(f"    {content['response']['Question']}\n\n")
                
                tex_file.write("    \\textbf{Solution} \n")
                tex_file.write(f"    {content['response']['Solution']}\n\n")
                
                # Add space between different questions and solutions
                tex_file.write("    \\vspace{1em}\n")

# Return the output LaTeX file path
output_tex_path
'''

'# Import the necessary libraries\nimport json\n\n# Load the restructured JSON file\ninput_file_path = \'/mnt/data/restructured_data_corrected.json\'\nwith open(input_file_path, \'r\') as file:\n    structured_data = json.load(file)\n\n# Define the output LaTeX file path\noutput_tex_path = \'/mnt/data/questions_output.tex\'\n\n# Open the LaTeX file for writing\nwith open(output_tex_path, \'w\') as tex_file:\n    # Iterate over the data to create the LaTeX structure\n    for kc_pair_key, topics in structured_data.items():\n        # Write the first layer: KCs as header 1\n        tex_file.write(f"\\section*{{{kc_pair_key}}}\n\n")\n        \n        for topic, temperatures in topics.items():\n            # Write the second layer: Topic as header 2\n            tex_file.write(f"  \\subsection*{{{topic}}}\n")\n            \n            for temperature, content in temperatures.items():\n                # Write the third layer: Temperature as header 3\n                tex_file.write(f"    \\

In [32]:
import ast
import os
from dotenv import load_dotenv
from openai import OpenAI
from string import Template

def question_evaluator(model, prompt, qid, grade, kc1, kc2, question, solution):
    load_dotenv()
    TOKEN = os.getenv("Openrouter_TOKEN")
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=TOKEN,
    )

    with open(prompt, 'r', encoding='utf-8') as file:
        template = file.read()

    template = Template(template)
    filled_prompt = template.safe_substitute(qid = qid, grade = grade, kc1 = kc1, kc2 = kc2, question = question, solution = solution)
    messages =  ast.literal_eval(filled_prompt)
    response = client.chat.completions.create(
        model=models[model], 
        messages=messages,
        temperature=0.5,
        n=1,
        max_tokens=10000,
        stop=None
    )
    response_text = response.choices[0].message.content
    response_dict = extract_dollars(response_text)
    if response_dict is None:
        response_dict = extract_json(response_text)
        if response_dict is None:
            try:
                response_dict = ast.literal_eval(response_text)
            except (ValueError, SyntaxError):
                with open("error.txt", 'a', encoding='utf-8') as file:
                    file.write(f"ID: {id}\n")
                    file.write(f"Prompt: {prompt}\n")
                    file.write(f"Response:\n{response_text}\n")
                    file.write(f"{'-'*40}\n")
                return None
    return response_dict

In [39]:
import json
import os

model = "qwen"
prompt = r"D:\AStar\Code\Prompts\GPT\evaluate_gpt.txt"
input_path = r"D:\AStar\Code\Data\Generation_multiple_KCs_steps\gpt_no_difficulty_2.json"
output_path = r"D:\AStar\Code\Data\Evaluation_multiple_optimized\qwen_no_difficulty_2.json"

with open(input_path, "r", encoding="utf-8") as f:
    questions = json.load(f)
if os.path.exists(output_path):
    with open(output_path, "r", encoding="utf-8") as f:
        all_responses = json.load(f)
else:
    all_responses = {}

for qid, qdata in questions.items():
    if qid in all_responses:
        continue 
    kc1 = qdata.get("kc1", "")
    kc2 = qdata.get("kc2", "")
    grade = qdata.get("Grade", "")
    question = qdata.get("Question", "")
    solution = qdata.get("Solution", "")
    response = question_evaluator(model=model, prompt=prompt, grade=grade, qid=qid, kc1=kc1, kc2=kc2, question=question, solution = solution)
    print(response)
    if response:
        all_responses[qid] = response
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(all_responses, f, indent=2, ensure_ascii=False)


{'ID': 'O1NumOps1O1NumOps21NoDif07', 'Score': {'Answerable': 1, 'Correctly aligned with the intended Knowledge Components': 1, 'Grammatically Correct': 1, 'Understandable': 1, 'Fluent, clear, and concise': 5, 'Realistic': 1, 'Correct LaTeX Format': 1}}
{'ID': 'O1NumOps1O1NumOps31NoDif06', 'Score': {'Answerable': 1, 'Correctly aligned with the intended Knowledge Components': 1, 'Grammatically Correct': 1, 'Understandable': 1, 'Fluent, clear, and concise': 5, 'Realistic': 1, 'Correct LaTeX Format': 1}}
{'ID': 'O1NumOps1O1NumOps41NoDif06', 'Score': {'Answerable': 1, 'Correctly aligned with the intended Knowledge Components': 1, 'Grammatically Correct': 1, 'Understandable': 1, 'Fluent, clear, and concise': 5, 'Realistic': 1, 'Correct LaTeX Format': 1}}
{'ID': 'O1NumOps1O1NumOps51NoDif06', 'Score': {'Answerable': 1, 'Correctly aligned with the intended Knowledge Components': 1, 'Grammatically Correct': 1, 'Understandable': 1, 'Fluent, clear, and concise': 4, 'Realistic': 1, 'Correct LaTeX F

In [42]:
import pandas as pd
import json

# Define the input paths for all model files
input_link = {
    "gpt": r"D:\AStar\Code\Data\Evaluation_multiple_mutual\gpt_no_difficulty_2.json",
    "claude": r"D:\AStar\Code\Data\Evaluation_multiple_mutual\claude_no_difficulty_2.json",
    "deepseek": r"D:\AStar\Code\Data\Evaluation_multiple_mutual\deepseek_no_difficulty_2.json",
    "phi": r"D:\AStar\Code\Data\Evaluation_multiple_mutual\phi_no_difficulty_2.json",
    "llama": r"D:\AStar\Code\Data\Evaluation_multiple_mutual\llama_no_difficulty_2.json",
    "qwen": r"D:\AStar\Code\Data\Evaluation_multiple_mutual\qwen_no_difficulty_2.json"
}

# Function to calculate average for each model file
def calculate_average(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Extract the scores from the data
    scores = []
    for key, value in data.items():
        scores.append(value["Score"])

    # Convert the scores to a DataFrame
    df = pd.DataFrame(scores)

    # Calculate the average of each criterion
    df_avg = df.mean()

    return df_avg

# Calculate the averages for each model
averages = {}
for model, path in input_link.items():
    averages[model] = calculate_average(path)

df_averages = pd.DataFrame(averages)
df_averages


,gpt,claude,deepseek,phi,llama,qwen
Answerable,1.0,1.000000,1.000000,1.000000,0.952381,1.000000
Correctly aligned with the intended Knowledge Components,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
Grammatically Correct,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
Understandable,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
"Fluent, clear, and concise",5.0,4.571429,5.000000,4.000000,4.809524,4.809524
Realistic,1.0,0.952381,0.904762,0.952381,1.000000,0.904762
Correct LaTeX Format,1.0,1.000000,0.904762,1.000000,1.000000,1.000000
